In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# Any results you write to the current directory are saved as output.

/kaggle/input/plant-pathology-2020-fgvc7/test.csv
/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv
/kaggle/input/plant-pathology-2020-fgvc7/train.csv
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_42.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_733.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_887.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1771.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1059.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1263.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_537.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_654.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1469.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_702.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_159.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_875.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_697.jpg
/kaggle/input/plant-pathology-20

/kaggle/input/plant-pathology-2020-fgvc7/images/Train_672.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1664.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1690.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_550.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_282.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1030.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1480.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1203.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_857.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_307.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1442.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_285.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1358.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1756.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1144.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_86.jpg
/kaggle

In [2]:
import cv2
import math
import numpy as np
import scipy as sp
import pandas as pd

import tensorflow as tf
from keras.layers import LeakyReLU



Using TensorFlow backend.


In [3]:

IMAGE_PATH = "../input/plant-pathology-2020-fgvc7/images/"
TEST_PATH = "../input/plant-pathology-2020-fgvc7/test.csv"
TRAIN_PATH = "../input/plant-pathology-2020-fgvc7/train.csv"
SUB_PATH = "../input/plant-pathology-2020-fgvc7/sample_submission.csv"

sub = pd.read_csv(SUB_PATH)
test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)

In [4]:
train_data.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [5]:
test_data.head()

,image_id
0,Test_0
1,Test_1
2,Test_2
3,Test_3
4,Test_4


In [6]:
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()

train_image=[]
SAMPLE_LEN = 100
img_size= 80

def load_image(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(IMAGE_PATH + file_path)
    image=cv2.resize(image,(img_size,img_size),interpolation=cv2.INTER_AREA)
    train_image.append(image)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

train_img = train_data["image_id"].progress_apply(load_image)

In [7]:
import plotly.express as px
fig = px.imshow(train_image[5])
fig.show()

In [8]:
test_image=[]
SAMPLE_LEN = 100
img_size= 80

def load_image(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(IMAGE_PATH + file_path)
    image=cv2.resize(image,(img_size,img_size),interpolation=cv2.INTER_AREA)
    test_image.append(image)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

test_img = train_data["image_id"].progress_apply(load_image)

In [9]:
fig = px.imshow(train_image[3])
fig.show()

In [10]:
X_Train = np.ndarray(shape=(len(train_image), img_size, img_size, 3),dtype = np.float32)
i=0
for image in train_image:
    #X_Train[i]=img_to_array(image)
    X_Train[i]=train_image[i]
    i=i+1
X_Train=X_Train/255
print('Train Shape: {}'.format(X_Train.shape))

Train Shape: (1821, 80, 80, 3)


In [11]:
X_Test = np.ndarray(shape=(len(test_image), img_size, img_size, 3),dtype = np.float32)
i=0
for image in test_image:
    #X_Test[i]=img_to_array(image)
    X_Test[i]=test_image[i]
    i=i+1
    
X_Test=X_Test/255
print('Test Shape: {}'.format(X_Test.shape))

Test Shape: (1821, 80, 80, 3)


In [12]:
y = train_data.drop(['image_id'],axis=1)
y.head()

,healthy,multiple_diseases,rust,scab
0,0,0,0,1
1,0,1,0,0
2,1,0,0,0
3,0,0,1,0
4,1,0,0,0


In [13]:
y_train = np.array(y.values)
y_train.shape

(1821, 4)

In [14]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten,BatchNormalization,LeakyReLU
from keras.optimizers import SGD
from keras.optimizers import Adam

In [ ]:
lr = LeakyReLU

In [21]:
def const():
        model = Sequential()
        
        
        model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', input_shape=(img_size, img_size, 3),
                activation='relu'))
        model.add(BatchNormalization()) # Normalize the activations of the previous layer at each batch
        model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu',kernel_initializer='he_normal'))
        model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
    
        model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu',kernel_initializer='he_normal'))
        model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
        model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
   
        model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu',kernel_initializer='he_normal'))
        model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
        model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Flatten()) # Flatten the input
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(32, activation='relu',kernel_initializer='he_normal'))
        model.add(Dense(4, activation='softmax'))
    # Configure the learning process
    # The loss function is the objective that the model will try to minimize
    # For any classification problem, use accuracy metric
        optimizer = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
        model.summary()
    
    
        return model


        


In [22]:
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau,ModelCheckpoint

In [23]:
X_train, X_val, Y_train, Y_val = train_test_split(X_Train, y_train, 
                                                  test_size=0.3, random_state=42)

In [24]:
model = const()
image_gen = ImageDataGenerator(rotation_range=360,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip = True,
                              vertical_flip = True)
image_gen.fit(X_train)

reduce = ReduceLROnPlateau(monitor='val_accuracy',factor=0.5,
                          patience=8,verbose=1,min_lr=1e-3)

checkp = ModelCheckpoint('model.h5',verbose=1,save_best_only=True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 80, 80, 32)        896       
_________________________________________________________________
batch_normalization_8 (Batch (None, 80, 80, 32)        128       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 80, 80, 32)        9248      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 80, 80, 32)        9248      
_________________________________________________________________
batch_normalization_9 (Batch (None, 80, 80, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 40, 40, 32)       

In [25]:
es = EarlyStopping(monitor='val_loss',patience=15)

In [ ]:
model.fit_generator(image_gen.flow(X_train, Y_train, batch_size=32),
            steps_per_epoch=X_train.shape[0] // 32,
            epochs=120,
            verbose=1,
            callbacks=[reduce,checkp],
            validation_data=(X_val, Y_val))

Epoch 1/120
39/39 [==============================] - 72s 2s/step - loss: 1.5075 - accuracy: 0.3084 - val_loss: 1.3431 - val_accuracy: 0.3346

Epoch 00001: val_loss improved from inf to 1.34311, saving model to model.h5
Epoch 2/120
39/39 [==============================] - 70s 2s/step - loss: 1.3326 - accuracy: 0.3591 - val_loss: 1.4352 - val_accuracy: 0.3309

Epoch 00002: val_loss did not improve from 1.34311
Epoch 3/120
39/39 [==============================] - 70s 2s/step - loss: 1.2947 - accuracy: 0.3502 - val_loss: 1.5709 - val_accuracy: 0.3400

Epoch 00003: val_loss did not improve from 1.34311
Epoch 4/120
39/39 [==============================] - 70s 2s/step - loss: 1.2665 - accuracy: 0.3639 - val_loss: 1.5834 - val_accuracy: 0.3254

Epoch 00004: val_loss did not improve from 1.34311
Epoch 5/120
39/39 [==============================] - 70s 2s/step - loss: 1.2571 - accuracy: 0.3760 - val_loss: 1.6549 - val_accuracy: 0.3382

Epoch 00005: val_loss did not improve from 1.34311
Epoch 6/1

39/39 [==============================] - 82s 2s/step - loss: 1.1224 - accuracy: 0.5008 - val_loss: 1.1580 - val_accuracy: 0.4570

Epoch 00041: val_loss improved from 1.16163 to 1.15795, saving model to model.h5
Epoch 42/120
39/39 [==============================] - 70s 2s/step - loss: 1.1197 - accuracy: 0.4992 - val_loss: 1.1330 - val_accuracy: 0.5137

Epoch 00042: val_loss improved from 1.15795 to 1.13299, saving model to model.h5
Epoch 43/120
39/39 [==============================] - 70s 2s/step - loss: 1.1213 - accuracy: 0.5064 - val_loss: 1.1566 - val_accuracy: 0.4662

Epoch 00043: val_loss did not improve from 1.13299
Epoch 44/120
39/39 [==============================] - 70s 2s/step - loss: 1.1111 - accuracy: 0.5008 - val_loss: 1.2101 - val_accuracy: 0.4369

Epoch 00044: val_loss did not improve from 1.13299
Epoch 45/120
39/39 [==============================] - 69s 2s/step - loss: 1.1203 - accuracy: 0.4984 - val_loss: 1.1453 - val_accuracy: 0.4662

Epoch 00045: val_loss did not impr

39/39 [==============================] - 68s 2s/step - loss: 0.5780 - accuracy: 0.7987 - val_loss: 1.6059 - val_accuracy: 0.5612

Epoch 00081: val_loss did not improve from 0.60015
Epoch 82/120
39/39 [==============================] - 69s 2s/step - loss: 0.5215 - accuracy: 0.8261 - val_loss: 0.5716 - val_accuracy: 0.7824

Epoch 00082: val_loss improved from 0.60015 to 0.57156, saving model to model.h5
Epoch 83/120
39/39 [==============================] - 69s 2s/step - loss: 0.5154 - accuracy: 0.8196 - val_loss: 1.7197 - val_accuracy: 0.6709

Epoch 00083: val_loss did not improve from 0.57156
Epoch 84/120
39/39 [==============================] - 68s 2s/step - loss: 0.5128 - accuracy: 0.8325 - val_loss: 0.7233 - val_accuracy: 0.7441

Epoch 00084: val_loss did not improve from 0.57156
Epoch 85/120
39/39 [==============================] - 68s 2s/step - loss: 0.4931 - accuracy: 0.8333 - val_loss: 0.6057 - val_accuracy: 0.8026

Epoch 00085: val_loss did not improve from 0.57156
Epoch 86/120


In [ ]:
loss = pd.DataFrame(model.history.history)
loss

In [ ]:
pred = model.predict(X_val)
pred

In [ ]:
accuracy = model.evaluate(X_val, Y_val, verbose=1)
print('Test_Accuracy:-', accuracy[1]*100)